In [ ]:
#generating a window
window_cnn = WindowGenerator(input_width=FORECAST_WINDOW+2, label_width=FORECAST_WINDOW, shift=FORECAST_SHIFT,
                             train_data=train_data, val_data=val_data, test_data=test_data, label_columns=TARGET_LABELS)
window_cnn

In [ ]:
for example_inputs, example_labels in window_cnn.train.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')

In [ ]:
def create_conv_model(input_shape=(FORECAST_WINDOW+2, 8)):
    inputs = kl.Input(shape=input_shape)
    conv1 = kl.Conv1D(filters=64, kernel_size=CONV_WIDTH, strides=1, activation="selu", padding="same")(inputs)
    pool1 = kl.MaxPool1D(pool_size=3, strides=1)(conv1)
    conv3 = kl.Conv1D(filters=128, kernel_size=3, strides=1, activation="selu", padding="same")(pool1)
    dense1 = kl.Dense(units=64)(conv3)
    dense1 = kl.PReLU()(dense1)
    dense1 = kl.Dropout(rate=0.3)(dense1)
    dense2 = kl.Dense(units=16)(dense1)
    dense2 = kl.PReLU()(dense2)
    dense2 = kl.Dropout(rate=0.3)(dense2)
    outputs = kl.Dense(units=len(TARGET_LABELS))(dense2)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer="adam", loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanAbsoluteError()])

    return model
conv_model = create_conv_model()
conv_model.summary()

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(patience=2)
history = conv_model.fit(window_cnn.train, epochs=10, validation_data=window_cnn.val, callbacks=[reduce_lr])

In [ ]:
plot_history(history.history)

In [ ]:
window_cnn.plot(conv_model, plot_col=TARGET_LABELS[2])